In [1]:
import json
import random

In [2]:
lang = "es"
diff = "easy"
length = 4

vocabs_pool: set[str] = set()
exclusive_pool: set[str] = set()
alphabets: set[str] = set()

with open(rf"vocabs\{lang}\{lang}-len{length}-{diff}.json") as file:
    json_data = json.load(file)
    for d in json_data:
        vocabs_pool.add(d)
        exclusive_pool.add(d)
        alphabets = alphabets.union(set(d))

# with open(r"vocabs\{lang}\{lang}-len{length}-medium.json") as file:
#     json_data = json.load(file)
#     for d in json_data:
#         exclusive_pool.remove(d)

num_vocabs = len(vocabs_pool)
print(len(alphabets), "".join(sorted(alphabets)))
print(f"{len(vocabs_pool)=}")
print(f"{len(exclusive_pool)=}")

32 abcdefghijklmnopqrstuvwxyzáéíñóú
len(vocabs_pool)=438
len(exclusive_pool)=438


In [3]:
def change_character(vocab: str) -> set[str]:
    new_vocabs = set()
    for i, c in enumerate(vocab):
        for a in alphabets:
            if a != c:
                new_vocab = vocab[:i] + a + vocab[i + 1 :]
                new_vocabs.add(new_vocab)

    return new_vocabs


print(change_character("abc"))

{'abí', 'abn', 'aqc', 'ayc', 'abó', 'abz', 'pbc', 'rbc', 'acc', 'ybc', 'atc', 'nbc', 'vbc', 'aic', 'avc', 'fbc', 'abe', 'wbc', 'dbc', 'zbc', 'ébc', 'aba', 'ábc', 'abd', 'sbc', 'abo', 'axc', 'awc', 'aoc', 'abq', 'auc', 'ibc', 'apc', 'abm', 'qbc', 'ubc', 'abú', 'aéc', 'ahc', 'aby', 'aec', 'íbc', 'abi', 'abx', 'afc', 'abf', 'abu', 'amc', 'abá', 'abñ', 'lbc', 'ajc', 'mbc', 'abp', 'abt', 'adc', 'azc', 'gbc', 'ñbc', 'abé', 'abl', 'abw', 'kbc', 'abh', 'abr', 'abb', 'ebc', 'akc', 'anc', 'agc', 'jbc', 'aúc', 'úbc', 'obc', 'aóc', 'aíc', 'xbc', 'hbc', 'abv', 'aác', 'bbc', 'abj', 'aac', 'arc', 'abg', 'abk', 'óbc', 'tbc', 'cbc', 'asc', 'abs', 'añc', 'alc'}


In [4]:
def character_difference(from_: str, to: str) -> int:
    count = 0
    for c1, c2 in zip(from_, to):
        if c1 != c2:
            count += 1
    return count

In [5]:
def distance(
    from_: str, to: str, max_distance: int | None = None
) -> tuple[int, list[str]]:
    vocab_path = {from_: ""}
    queue = [(from_, 0)]
    while queue:
        (cur_vocab, cur_distance) = queue.pop(0)
        if cur_vocab == to:
            path = [cur_vocab]
            while path[0] != from_:
                path.insert(0, vocab_path[path[0]])
            return (cur_distance, path)

        if cur_distance == max_distance:
            continue

        for new_vocab in change_character(cur_vocab):
            if (new_vocab not in vocab_path) and (new_vocab in vocabs_pool):
                vocab_path[new_vocab] = cur_vocab
                queue.append((new_vocab, cur_distance + 1))

    return (-1, [])

In [6]:
print(distance("admin", "admit"))
print(distance("line", "firm"))
print(distance("war", "him"))
print(distance("センセイ", "モンダイ"))
print(distance("caro", "mano"))

(-1, [])
(-1, [])
(-1, [])
(-1, [])
(-1, [])


In [7]:
# easy: {"max_distance": 4, "min_distance": 3, "strict": False}

max_distance = 4
strict = False
min_distance = 3

count = 0
count_exclusive = 0
starter = set()
involved = set()
pairs: list[tuple[str, str]] = []

for i, v1 in enumerate(vocabs_pool):
    for j, v2 in enumerate(vocabs_pool):
        dist = distance(v1, v2, max_distance=max_distance)
        if strict:
            difference = character_difference(v1, v2)
            flag = dist[0] > difference and dist[0] >= min_distance
        else:
            flag = dist[0] >= min_distance

        if flag:
            count += 1
            starter.add(v1)
            involved = involved.union(set(dist[1]))
            # print(dist)

            if any(v in exclusive_pool for v in dist[1]):
                count_exclusive += 1
                pairs.append((v1, v2))

    if i % (num_vocabs // 10) == 0:
        print(f'checking "{v1}"| {i}/{num_vocabs}')

total_combinations = num_vocabs * (num_vocabs - 1)

pair_rate = count / total_combinations
print(f"pair rate: {count}/{total_combinations} = {pair_rate:.2%}")

exclusive_rate = count_exclusive / total_combinations
print(
    f"exclusive pair rate: {count_exclusive}/{total_combinations} = {exclusive_rate:.2%}"
)

starter_rate = len(starter) / num_vocabs
print(f"starter: {len(starter)}/{num_vocabs} = {starter_rate:.2%}")

participation_rate = len(involved) / num_vocabs
print(f"participation: {len(involved)}/{num_vocabs} = {participation_rate:.2%}")

checking "opep"| 0/438
checking "hubo"| 43/438
checking "paul"| 86/438
checking "aviv"| 129/438
checking "área"| 172/438
checking "lima"| 215/438
checking "bajó"| 258/438
checking "ello"| 301/438
checking "japn"| 344/438
checking "jean"| 387/438
checking "caos"| 430/438
pair rate: 7038/191406 = 3.68%
exclusive pair rate: 7038/191406 = 3.68%
starter: 247/438 = 56.39%
participation: 253/438 = 57.76%


In [8]:
random.seed(0)
pairs_shuffled = pairs[:]
random.shuffle(pairs_shuffled)
with open(rf"vocabs\{lang}\{lang}-len{length}-pairs-{diff}.json", "w") as f:
    json.dump(pairs_shuffled, f)

for i in range(5):
    pair = pairs_shuffled[i]
    dist = distance(pair[0], pair[1])
    print(dist)

(4, ['visa', 'misa', 'masa', 'casa', 'cosa'])
(3, ['gama', 'dama', 'daba', 'haba'])
(3, ['rama', 'lama', 'lara', 'para'])
(4, ['vive', 'viva', 'visa', 'misa', 'mira'])
(4, ['mata', 'mala', 'gala', 'gama', 'lama'])


In [9]:
def create_distance_graph():
    distance_graph: dict[tuple[str, str], tuple[int, list[str]]] = {}

    def calculate_distance(from_: str, to: str):
        if (from_, to) in distance_graph:
            return

        min_distance = -1
        min_path = []
        for new_vocab in change_character(from_):
            if new_vocab in vocabs_pool:
                calculate_distance(new_vocab, to)
                distance, path = distance_graph[(new_vocab, to)]
                if min_distance == -1 or 0 <= distance < min_distance:
                    min_distance = distance + 1
                    min_path = [from_] + path
        distance_graph[(from_, to)] = (min_distance, min_path)

    for v1 in vocabs_pool:
        for v2 in vocabs_pool:
            calculate_distance(v1, v2)
    return distance_graph


# distance_graph = create_distance_graph()
# print(distance_graph)